In [1]:
from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *
from sklearn.preprocessing import LabelEncoder
from typing import Dict,Tuple,List
import conf
from UI.LOG import *
import cv2
from aml.train_pipeline import *  
from aml.train_pipeline import *
import pandas as pd

import os
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor,FasterRCNN_ResNet50_FPN_Weights
from torchinfo import summary

# from torchvision.models.detection import ssd300_vgg16,SSD300_VGG16_Weights
from torchvision.models.detection.ssd import SSDClassificationHead,SSD300_VGG16_Weights,det_utils
from torchvision.models.detection import ssd300_vgg16

import torch

import aml.model_using as model_using
import aml.support_func as support_funcS
import aml.time_mesuarment as time_mesuarment
import sys

import aml.managers as managers
import aml.img_processing as img_processing
import random
import numpy as np
import pprint
from torchinfo import summary
from aml.img_processing import *

from PIL import Image
import aml.models as models
import matplotlib.pyplot as plt
from pprint import pprint as Print
from PIL import Image
import warnings
from torchvision.utils import draw_bounding_boxes  
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
from torchvision.ops import nms 
from torchmetrics.detection.mean_ap import MeanAveragePrecision as mAP
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
from IPython.display import IFrame, display, HTML
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import catboost
from io import StringIO 
import sys
from sklearn.metrics import roc_auc_score
from catboost import utils
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from tqdm import tqdm
import multiprocessing as mp
import lightgbm as lgb
from numba import jit
from aml.pandas_utils import *
tqdm.pandas()

def simple_plot(x,y,title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y)
    ax.set_title(title)
    ax.grid()

    return fig,ax


def simple_plot_y1y2(x,y1,y2,label1='',label2='',title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y1,label=label1)
    ax.plot(x,y2,label=label2)
    ax.set_title(title)
    ax.grid(which='both')
    ax.legend()
    return fig,ax

def simple_plot_y1y2y3y4(x,y1,y2,y3,y4,label1='',label2='',label3='',label4='',title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y1,label=label1)
    ax.plot(x,y2,label=label2)
    ax.plot(x,y3,label=label3)
    ax.plot(x,y4,label=label4)
    ax.set_title(title)
    ax.grid(which='both')

    ax.legend()
    return fig,ax


def Gini(fpr,tpr):
    return 2*metrics.auc(fpr,tpr)-1
def plot_gxb_train_results(results):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(results['train-auc-mean'],label= 'train-auc-mean')
    ax.plot(results['test-auc-mean'],label= 'test-auc-mean')
    ax.legend()
    return fig,ax


def xgb_fit(X,Y, meta_of_fit, path_to_save_model,params,verbose=False):
    timer = time_mesuarment.Timer()
    timer.start()
    eval_metric = params['eval_metric']
    model = XGBClassifier(**params)
    # print('start model fit')

    # X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, train_size=0.8, random_state=1234)
    # model.fit(X_train,Y_train, eval_set=[(X_train,Y_train),(X_validation,Y_validation)],verbose=False)

    model.fit(X,Y, eval_set=[(X,Y)],verbose=verbose)

    # print('train end')
    # feature_important = model.get_booster().get_score(importance_type='weight')
    # keys = list(feature_important.keys())
    # values = list(feature_important.values())
    # argsort_ = np.argsort(values)
    # print('feauture importance')
    # for ind in argsort_:
    #     print('{} {}'.format(keys[ind],values[ind]))
    results = model.evals_result()
    train_loss = np.array(results['validation_0'][eval_metric])
    # val_loss = np.array(results['validation_1'][eval_metric])
    timer.stop()
    print('"{}" last train {} {} time_of_train {} min'.format(
        meta_of_fit,eval_metric,train_loss[-1],str(timer.get_execution_time()/60)))
    model.save_model(fname=path_to_save_model)
    


def train_model(X,Y, meta_of_fit, path_to_save_model, params,verbose):
    # wrap train to another process
    try:
        target_ = mp.Process(target=xgb_fit,args=(X,Y, meta_of_fit, path_to_save_model, params, verbose))
        target_.start()
        target_.join()
    except:
        print('somth went wrong')
        raise SystemExit

@jit(nopython = True)
def get_index_of_time_interval(t, time_intervals):
    for i in range(len(time_intervals)):
        t1t2 = time_intervals[i]
        if t >= t1t2[0] and t < t1t2[1]:
            return i
        if i == len(time_intervals)-1:
            if t >= t1t2[0] and t <= t1t2[1]:
                return i

@jit(nopython=True)
def set_values_to_buffer_by_indexes(buffer,indexes,values):
    for i in range(len(indexes)):
        buffer[indexes[i]] = values[i]

class TimeModel:
    models: List[XGBClassifier]
    time_intervals: np.array
    def __init__(self,path_pattern, time_intervals):
        self.models = []
        self.time_intervals = time_intervals
        for i in range(len(time_intervals)):
            model_i = XGBClassifier()
            model_i.load_model(fname=path_pattern+str(i)+'.json')
            self.models.append(model_i)

    def predict(self, X: pd.DataFrame):
        groups_by_time = group_by_time_interval(X,'issue_d',self.time_intervals)
        labels = np.zeros(shape=(X.shape[0],), dtype=np.intc)
        probas = np.zeros(shape=(X.shape[0],2), dtype=np.float64)
        i_ = 0
        for time_interval in self.time_intervals:
            print(time_interval)
            X_ = groups_by_time[i_]
            indexes_ = X_.index.values
            predict_ = self.models[i_].predict(X_)
            probas_ = self.models[i_].predict_proba(X_)
            set_values_to_buffer_by_indexes(labels,indexes_,predict_)
            set_values_to_buffer_by_indexes(probas,indexes_,probas_)
            i_+=1
        return labels, probas



In [2]:
X = pd.read_csv(conf.X_train_dataset,index_col=False)
Y = pd.read_csv(conf.Y_train_dataset,index_col=False)

In [3]:
t_1,t_2 = get_min_max(X,'issue_d')
delta_t =  3.0
N = int((t_2-t_1)/delta_t)+1
time_bins = np.linspace(start=t_1,stop= t_2,num=N)
time_intervals = [(time_bins[i],time_bins[i+1]) for i in range(len(time_bins)-1)]
# plot_float_distribution(X_train['issue_d'],(16,9),'issue date density')
groups_by_time = group_by_time_interval(X,'issue_d',time_intervals)

3/3


In [6]:
print(time_intervals)

[(2007.495867768595, 2010.8078512396694), (2010.8078512396694, 2014.1198347107438), (2014.1198347107438, 2017.4318181818185), (2017.4318181818185, 2020.743801652893)]


In [8]:
# N 2658
# "(2007.495867768595, 2008.5149396058487)" last train auc  0.8891242996386302 time_of_train 0.01614310344060262 min
# N 5750
# "(2008.5149396058487, 2009.5340114431024)" last train auc 0.9454247130436597 time_of_train 0.04041525920232137 min
# N 18640
# "(2009.5340114431024, 2010.553083280356)" last train auc  0.9264374299164312 time_of_train 0.11836928129196167 min
# N 38865
# "(2010.553083280356, 2011.5721551176098)" last train auc  0.8507712342316202 time_of_train 0.11460882822672526 min
# N 85573
# "(2011.5721551176098, 2012.5912269548635)" last train auc 0.7977982670191832 time_of_train 0.24786526759465535 min
# N 231228
# "(2012.5912269548635, 2013.6102987921172)" last train auc 0.7610506682236307 time_of_train 0.624234139919281 min
# N 488014
# "(2013.6102987921172, 2014.6293706293707)" last train auc 0.7587616124137769 time_of_train 4.1776900370915735 min
# N 785743
# "(2014.6293706293707, 2015.6484424666244)" last train auc 0.7399517215244317 time_of_train 5.5209632833798725 min
# N 1108488
# "(2015.6484424666244, 2016.667514303878)" last train auc  0.7593030440503921 time_of_train 11.677052640914917 min
# N 790043
# "(2016.667514303878, 2017.6865861411318)" last train auc  0.7522980749334666 time_of_train 6.250174677371978 min
# N 494389
# "(2017.6865861411318, 2018.7056579783855)" last train auc 0.7596001844717735 time_of_train 3.1340223789215087 min
# N 241778
# "(2018.7056579783855, 2019.7247298156392)" last train auc 0.7939667284221041 time_of_train 1.3382912317911784 min
# N 35946
# "(2019.7247298156392, 2020.743801652893)" last train auc  0.9027145554153396 time_of_train 0.11624847253163656 min
# n_est_by_time_interval_index = [
    
# ]
# max_depth_vec = [
#     5,  5,   5,  5, 6 , 7,  7, 7, 7, 7, 7, 6, 5
# ]
# if len(n_est_by_time_interval_index) != len(time_intervals):
#     raise SystemExit
# if len(max_depth_vec) != len(time_intervals):
#     raise SystemExit

for i in range(len(time_intervals)):
    ith_time_interval = time_intervals[i]
    X_ = groups_by_time[i]
    Y_ = Y.loc[X_.index]
    # n_estimators = n_est_by_time_interval_index[i]
    # max_depth = max_depth_vec[i]
    N = len(Y_)
    # min_child_weight = 0
    eval_metric = 'auc'
    # if N < 10:
    #     min_child_weight = 1
    # if N>= 10**1 and N < 10**2:
    #     min_child_weight = 3
    # if N>= 10**2 and N < 10**3:
    #     min_child_weight = 5
    # if N>= 10**3 and N < 10**4:
    #     min_child_weight = 10
    # if N >= 10**4 and N < 10**5:
    #     min_child_weight = 20
    # if N >= 10**5:
    #     min_child_weight = 30
    print('N {}'.format(N))

    params = {
            'eta': 0.01,
            'max_depth': 8,
            'n_estimators': 4000,
            'gamma':  0.2,
            'colsample_bytree':0.5, 
            'colsample_bylevel':0.5, 
            'colsample_bynode':0.5,
            'lambda': 1.5,
            'alpha': 0.1,
            'scale_pos_weight': np.sum(np.where(Y==0))/np.sum(np.where(Y==1)),
            'min_child_weight': 5,
            'max_delta_step': 5,
            'objective':'binary:logistic',
            'eval_metric': eval_metric, 
            'tree_method':'gpu_hist',
            'n_jobs':8,
            'gpu_id':0,
            'seed':0,
            'subsample': 0.9,
    }
    train_model(X=X_,Y=Y_,meta_of_fit=str(ith_time_interval),path_to_save_model=conf.xgb_models_pattern+str(i)+'.json',params=params,verbose=False)

N 9673
"(2007.495867768595, 2010.8078512396694)" last train auc 0.9999702299136234 time_of_train 0.4863743225733439 min
N 153571
"(2010.8078512396694, 2014.1198347107438)" last train auc 0.8844455011462522 time_of_train 3.103721233208974 min
N 773797
"(2014.1198347107438, 2017.4318181818185)" last train auc 0.7826874402433004 time_of_train 26.615129880110423 min
N 262820
"(2017.4318181818185, 2020.743801652893)" last train auc 0.8442578570939 time_of_train 13.84238266547521 min


In [9]:
model = TimeModel(path_pattern=conf.xgb_models_pattern,time_intervals= time_intervals)
eval_preds_,eval_proba_ = model.predict(X)

3/3
(2007.495867768595, 2010.8078512396694)
(2010.8078512396694, 2014.1198347107438)
(2014.1198347107438, 2017.4318181818185)
(2017.4318181818185, 2020.743801652893)


In [14]:
@jit(nopython=True)
def apply_limits(probas):
    o_ = np.zeros(shape=(len(probas),))
    for i in range(len(probas)):
        if probas[i]>0.999:
            o_[i]= 0.999
            continue
        if probas[i] < 0.001:
            o_[i] = 0.001
            continue
        o_[i] = probas[i]
    return o_

In [16]:
fpr,tpr,tr = metrics.roc_curve(Y,eval_proba_[:,1],pos_label=1)
# simple_plot(fpr,tpr,title='ROC XGBoost')
print('AUC {}'.format(metrics.auc(fpr,tpr)))
print('Gini {}'.format(Gini(fpr,tpr)))

AUC 0.8130930423869495
Gini 0.6261860847738989


In [11]:
X_eval = pd.read_csv(conf.X_test_dataset,index_col=False)
submit_preds_,submit_proba_ = model.predict(X_eval)

3/3
(2007.495867768595, 2010.8078512396694)
(2010.8078512396694, 2014.1198347107438)
(2014.1198347107438, 2017.4318181818185)
(2017.4318181818185, 2020.743801652893)


In [12]:
def make_submission(positive_probabilities):
    d = {'index': [i for i in range(0,len(positive_probabilities))], 'loan_status': positive_probabilities}
    o_df = pd.DataFrame(data=d)
    submission_csv = os.path.join('answer.csv')
    submission_zip = os.path.join(conf.data_folder,'answer.zip')
    o_df.to_csv(submission_csv,index=False)
    os.system('rm {}'.format(submission_zip))
    os.system('zip {} {}'.format(submission_zip, submission_csv))
make_submission(positive_probabilities=submit_proba_[:,1])

  adding: answer.csv (deflated 57%)
